# Project Phase 2 
## Improving DeepDTA







## 1. Analysis of DeepDTA Model

### 1.1 Positive Points:

### 1.2 Negative Points:

## 2. Resolving the Negative Points

## 3. Preprocessing

## 4. Implementation

### 4.1. Setup 

#### 4.1.1 Importing libraries

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#### 4.1.2 Loading data

## 5. Evaluation

## 6. Distribution Histogram

## 7. Choosing the best model

## 8. Interpreting the model

### 8.1. Saliency Maps

### 8.2. Guided Back Propagation

### 8.3. Guided Grad-Cam

### 8.4. LRP

## 9. Robustness & Accuracy

## 10. Meaning of Repetitions

## 11. Making the model more interpretable